**MARKET BASKET ANALYSIS NOTEBOOK**

Dowload and preprocess dataset

In [8]:
!pip install kaggle
!pip install pandas

import os,zipfile,json,re
import functools as ft

os.environ['KAGGLE_USERNAME']='giacomoturati1'
os.environ['KAGGLE_KEY']='7d34a1aefc3558065164b70c24ce27ed'

from kaggle.api.kaggle_api_extended import KaggleApi

def get_dataset():

  #execute only if the dataset was not already downloaded
  if 'old-newspaper.tsv' not in os.listdir():
    api=KaggleApi()
    api.authenticate()

    api.dataset_download_file('alvations/old-newspapers','old-newspaper.tsv')

    with zipfile.ZipFile('old-newspaper.tsv.zip','r') as _zip:
      _zip.extractall()

#languages: subset of languages to be considered during the market-basket analysis
def read_dataset_from_disk(languages=None):
  with open('old-newspaper.tsv','r') as f:
    #skip header line
    next(f)
    for line in f:
      l=line.split('\t')
      if languages is not None and l[0] not in languages: continue
      yield l

def create_test_json_dataset(languages=None):
  baskets=[]

  filename=ft.reduce(lambda i,j:i+'_'+j,languages) if languages is not None else "all_languages"
  filename+='.json'

  #execute only if the dataset was not already created
  if filename not in os.listdir():
    for line in read_dataset_from_disk(languages):
      #get a list of words as basket skipping any sequence of non-alphabetical characters
      basket=re.split(r'[^a-zA-Z]+',line[3])
      #remove any empty string
      basket=[word.lower() for word in basket if word!='']
      baskets.append(basket)

    f=open(filename,'w')
    f.write(json.dumps(baskets,indent="\t"))
    f.close()

def iter_baskets_from_disk(filename):
  with open(filename,'r') as f:
    #skip first square braket
    next(f)
    basket=[]
    


get_dataset()
create_test_json_dataset(['Italian'])




In [9]:
count=0
with open('Italian.json','r') as f:
  for line in f:
    print(line)
    count+=1
    if count==15: break


[

	[

		"con",

		"l",

		"editore",

		"crocetti",

		"il",

		"poeta",

		"e",

		"drammaturgo",

		"wole",

		"soyinka",

		"premio",

		"nobel",

		"per",

